# Юнит 2

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv('wineQualityReds.zip')

In [4]:
#ЗАДАНИЕ 2.7

df['target']= np.where(df.quality >=6,1,0)
Y= df['target']
X = df.drop(['target', 'quality'], axis= 1)
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
print(f'LogisticRegression f1_score: {f1_score(preds_test, y_test)}')
model = DecisionTreeClassifier(max_depth = 10, random_state=42)
model.fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
print(f'DecisionTreeClassifier f1_score: {f1_score(preds_test, y_test)}')

LogisticRegression f1_score: 0.7390476190476191
DecisionTreeClassifier f1_score: 0.7601476014760148


c:\Users\AubakirovMA\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
#ЗАДАНИЕ 2.8
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth = 10, random_state=42), n_estimators=1500, random_state=42).fit(X_train, y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
f1_score(preds_test, y_test)

c:\Users\AubakirovMA\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.8239700374531835

# Юнит 3

In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

data = pd.read_csv('boston__1_.zip', decimal=',')

In [13]:
#ЗАДАНИЕ 3.4

#1.

X = data[data.columns[:-1]]
Y = data["target"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 13)
print(f'Mедианная цена: {Y_train.mean()}')

#2.

model_lr = LinearRegression()
model_lr.fit(X_train, Y_train)
preds_train = model_lr.predict(X_train)
preds_test = model_lr.predict(X_test)
print(f'LinearRegression MAE: {mean_absolute_error(Y_test, preds_test)}')

#3.

model = DecisionTreeRegressor(random_state=13)
model.fit(X_train, Y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)
print(f'DecisionTreeRegressor train MAE: {mean_absolute_error(Y_train, preds_train)}')
print(f'DecisionTreeRegressor test MAE: {mean_absolute_error(Y_test, preds_test)}')

#Вывод о переобучении делаем на основании того, что ошибка на обучении нулевая и имеет большой разрыв с ошибкой на тестовой выборке.

#4.

for n in [3, 10, 100, 500]:
    model = RandomForestRegressor(n_estimators=n, random_state = 13)
    model.fit(X_train, Y_train)
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)
    print(f'MAE for {n} is {mean_absolute_error(Y_test, preds_test)}')

Mедианная цена: 22.77316384180791
LinearRegression MAE: 3.722279395856146
DecisionTreeRegressor train MAE: 0.0
DecisionTreeRegressor test MAE: 2.8388157894736845
MAE for 3 is 2.9328947368421057
MAE for 10 is 2.466315789473684
MAE for 100 is 2.2559736842105265
MAE for 500 is 2.2365657894736892


# Юнит 4

In [28]:
import pandas as pd
import numpy as np

df = pd.read_csv('weatherAUS.zip')

In [29]:
# ЗАДАНИЕ 4.1

df.isnull().sum().sum()

343248

In [30]:
# ЗАДАНИЕ 4.2

round(df.isna().sum() / len(df), 3)
df.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)
df.shape[0]

145460

In [32]:
# ЗАДАНИЕ 4.3

df.RainToday = df.RainToday.map({'No': 0, 'Yes': 1})
df.RainTomorrow = df.RainTomorrow.map({'No': 0, 'Yes': 1})
df.RainToday.mean()

0.22419285648984874

In [33]:
# ЗАДАНИЕ 4.4

df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.dt.month
df.drop('Date', axis = 1, inplace = True)
df_season = df.groupby('Month').mean()
df_season[['RainToday']]

C:\Users\AubakirovMA\AppData\Local\Temp\ipykernel_17952\3201334474.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_season = df.groupby('Month').mean()


,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


In [34]:
# ЗАДАНИЕ 4.5

categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(df, columns=categoricals)
df_dummies.shape

(145460, 124)

In [35]:
# ЗАДАНИЕ 4.6

df_dummies.dropna(inplace=True)
X = df_dummies.drop('RainTomorrow', axis = 1)
Y = df_dummies['RainTomorrow']  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 31)
Y_test.mean()

0.22770253002811142

In [36]:
# ЗАДАНИЕ 4.7

def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.02879072820657669

In [37]:
# ЗАДАНИЕ 4.8
from sklearn.metrics import roc_auc_score

clf = LogisticRegression()
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
roc_auc_score(Y_test, preds_test)

c:\Users\AubakirovMA\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7287664187528149

In [38]:
# ЗАДАНИЕ 4.9

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    'max_leaf_nodes': list(range(2, 10)), 
    'min_samples_split': [2, 3, 4], 
    'max_depth': [5,7,9,11]
}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)

clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

In [39]:
# ЗАДАНИЕ 4.10

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

0.73


In [40]:
# ЗАДАНИЕ 4.11

params = {
    'max_features': [4, 5, 6, 7], 
    'min_samples_leaf': [3, 5, 7, 9, 11], 
    'max_depth': [5, 10, 15]
}
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)
clf = RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   1.1s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   1.1s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   1.1s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   1.2s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   1.1s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   1.1s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   1.2s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.1s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.1s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   1.1

In [41]:
# ЗАДАНИЕ 4.12

feature_names = [x for x in df_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000


# Юнит 6

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('AirPass.zip')

In [4]:
# ЗАДАНИЕ 6.1

df.isnull().sum().sum()

310

In [5]:
# ЗАДАНИЕ 6.2

df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
df['Arrival Delay in Minutes'].mean()

15.133392362180475

In [6]:
# ЗАДАНИЕ 6.3

print(df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0])

print(df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0])

print(df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0])

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: satisfaction, dtype: float64
Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: satisfaction, dtype: float64
Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64


In [7]:
# чтобы сработало 
 
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [8]:
# ЗАДАНИЕ 6.4

df=pd.get_dummies(df)
df.shape

(103904, 27)

In [9]:
# хз откуда взялась

df = df.drop('Unnamed: 0', axis=1)

In [10]:
# ЗАДАНИЕ 6.5
from sklearn.model_selection import train_test_split

X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

In [11]:
# ЗАДАНИЕ 6.6

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

In [12]:
# ЗАДАНИЕ 6.7

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
f1_score(preds_test, y_test)

0.8546883773161146

In [13]:
# ЗАДАНИЕ 6.8

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
f1_score(preds_test, y_test)

0.9404794558121674

In [14]:
# ЗАДАНИЕ 6.9 (~5.30m)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(f1_score),
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.2s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.4s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.4s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.0s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.0s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [15]:
# ЗАДАНИЕ 6.10

from xgboost import XGBClassifier

model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
f1_score(preds_test, y_test)

0.9579785161685312

In [ ]:
# ЗАДАНИЕ 6.11

from catboost import CatBoostClassifier, Pool

model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
f1_score(preds_class, y_test)

# 0.9598221728257851

In [1]:
# ЗАДАНИЕ 6.12

from catboost.utils import get_confusion_matrix

cm = get_confusion_matrix(model, Pool(X_train, y_train))

# array([[46668.,   538.],
#       [ 1255., 34662.]])

ModuleNotFoundError: No module named 'catboost'

In [16]:
# ЗАДАНИЕ 6.13

pd.DataFrame(
    {
        "feature_importance": clf.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

#

NameError: name 'clf' is not defined

***

In [17]:
df = pd.read_csv('Frogs_MFCCs.zip')
X = df.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
y = df['Family'].apply(lambda x: 1 if x=='Dendrobatidae' else 0)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=31)
y_test.shape

(1439,)

In [19]:
# ЗАДАНИЕ 7.5

from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(10, random_state=42)
model_rfc.fit(X_train, y_train)
pred = model_rfc.predict(X_test)
f1_score(y_test, pred)

0.9726775956284154

In [20]:
def is_numpy_c_ordered(X):
    return hasattr(X, "flags") and X.flags.c_contiguous

In [21]:
# ЗАДАНИЕ 7.6

from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=31)),
              ('knn', KNeighborsClassifier(n_neighbors= 11)),
              ('nb', GaussianNB() )               
]
metamodel = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
metamodel.fit(X_train, y_train)
pred = metamodel.predict(X_test)
f1_score(y_test, pred)

0.989247311827957